In [ ]:
import re
import os
import numpy as np
import pandas as pd
import seqlogo #This package can be found in the Conda repository bioconda/packages/seqlogo 

In [ ]:
import_directory = os.fsencode('./all_ap_dimersfiles/')

In [ ]:
def get_residues2(knob_residues, dictionary_residtoheptad):

    """Takes all lines that include information about knob residues and returns two dictionaries one for a and one for d
    knobs where the values contain lists. Within lists the first entry is a knob and subsequent four hole residues are as follows:
    a =  aNterm, d, e, aCterm
    d = dNterm, g, a, dCterm"""

    one_letter_code = {'A':'ALA','C':'CYS','D':'ASP','E':'GLU','F':'PHE','G':'GLY','H':'HIS','I':'ILE','K':'LYS',
                        'L':'LEU','M':'MET','N':'ASN','P':'PRO','Q':'GLN','R':'ARG','S':'SER','T':'THR','V':'VAL'               
                        ,'W':'TRP','Y':'TYR'}

    three_letter_code = {'ALA':'A','CYS':'C','ASP':'D','GLU':'E','PHE':'F','GLY':'G','HIS':'H','ILE':'I','LYS':'K',
                        'LEU':'L','MET':'M','ASN':'N','PRO':'P','GLN':'Q','ARG':'R','SER':'S','THR':'T','VAL':'V'               
                        ,'TRP':'W','TYR':'Y'}

    all_knobs = []
    for knob_residue_line in knob_residues:
        #this section splits and formats a line: remove brackets, colons, commas and new line characters 
        split_line = re.split(' ',knob_residue_line)
        split_line_nochar = []
        for x in split_line: 
            split_line_nochar.append(x.replace(')','').replace('(','').replace(':','').replace(',','').replace('\n',''))
        #this extracts each knob residue and hole residues as residuenumber_chainletter in a list
        one_knob_entry = []
        aa_counter = 1
        translate_counter = 1
        for x in split_line_nochar:
            if x[-2:-1].isdigit() and not x[-1:].isdigit() and translate_counter==1:
                one_knob_entry.append(x)
                translate_counter += 1
            elif x in three_letter_code and aa_counter ==1:
                aa_counter += 1
            elif x in three_letter_code:
                one_knob_entry.append(three_letter_code[x])
            else:
                pass
        all_knobs.append(one_knob_entry)
    #translate knoblist takes residuenumber_chainletter and coverts to amino acid and heptad position
    #it also filters out knob residues and holes that contain non-canonical amino acids
    translate_knoblist = []
    for knob_entry in all_knobs:
        if knob_entry[0] not in dict_residtohept:
            pass
        elif len(knob_entry) != 5:
            pass
        else:
            translated_knob_entry = []
            for individual_residue in knob_entry:
                if individual_residue not in dict_residtohept:
                    translated_knob_entry.append(individual_residue)
                else:
                    translated_knob_entry.append(dict_residtohept[individual_residue])
            translate_knoblist.append(translated_knob_entry)

    a_residues = {}
    d_residues = {}
    a_counter = 1
    d_counter = 1
    for KIH_info in translate_knoblist:
        if KIH_info[0][1] == 'a':
            a_residues['a'+str(a_counter)]=[x[0] for x in KIH_info]
            a_counter +=1
        if KIH_info[0][1] == 'd':
            d_residues['d'+str(d_counter)]=[x[0] for x in KIH_info]
            d_counter +=1
    return a_residues, d_residues

def dictionary_residtoheptad2(helices_info):
    """iterate through lines of socket output to generate a dictionary with residuenumber_chainname as key and
    a list of amino acid and heptad position as output"""
    helix_info_dict = {}
    for line in helices_info:
        if line[0:6] == 'assign':
            a = [int(s) for s in re.split('-| |:',line) if s.isdigit()]
            a = a[1:3]
            chain_name = line[-2:-1]
        elif line[0:6] == 'sequen':
            sequence = line[9:-1]
        elif line[0:6] == 'regist':
            register = line[9:-1]
            character_number = 0
            for sequence_value, register_value in zip(sequence,register):
                #This is to catch PDB files that start with negative numbers, those that cannot be resolved by this
                #are numbered from 1 onwards and will not match when called by get_residues() and subsequently
                #will be discarded.
                if len(range(a[0],a[1]+1)) == len(sequence):
                    residue_number = range(a[0],a[1]+1)[character_number]
                elif len(range(-1*a[0],a[1]+1)) == len(sequence):
                    residue_number = range(-1*a[0],a[1]+1)[character_number]
                else:
                    residue_number = 'xxx'
                helix_info_dict[str(residue_number)+chain_name]=[sequence_value,register_value]
                character_number += 1
        else:
             pass
    return helix_info_dict

def CountFrequency(my_list): 
      
    # Creating an empty dictionary  
    freq = {} 
    for items in my_list: 
        freq[items] = my_list.count(items) 
     
    return freq

def convert_to_probability(array_all):
    array_all_outof1 = {}
    total_sum = {}
    #Converts each row to a total of 1
    for complete_entry in array_all:
        frequence_array = np.zeros((4,20))
        row_number = 0
        for row in array_all[complete_entry]:
            divide_by = np.ones(20)*np.sum(row)
            frequency_row = np.true_divide(row, divide_by, out=np.zeros_like(row), where=divide_by!=0)
            frequence_array[row_number]= frequency_row
            row_number += 1
        total_sum[complete_entry]=divide_by[0]
        array_all_outof1[complete_entry]=frequence_array

    return array_all_outof1, total_sum

In [ ]:
##Execute code to get the two (a & d knob residues) dictionaries
all_a_dictionary = {}
all_d_dictionary = {}
for socket_file in sorted(os.listdir(import_directory)):
    filename = os.fsdecode(socket_file)
    directory = os.fsdecode(import_directory)
    if os.path.isfile(directory+filename) and not filename.startswith('.') :
        with open(directory+filename, 'r') as socket_output:
                ##numbering will differ from input as blank lines are stripped
                socket_lines = [line for line in socket_output.readlines() if line.strip()]
                ##extract lines that record knob residues
                knob_residues = []
                for line in socket_lines:
                    if len(re.split(' ',line)[0]) < 2:
                        pass
                    elif re.split(' ',line)[0][-2].isdigit() and re.split(' ',line)[0][-1] == ')':
                        knob_residues.append(line)
                #get lines that will identify heptad position on helix
                helices_info = [x for x in socket_lines if x[0:6] == 'assign' or x[0:6] == 'sequen' or x[0:6] == 'regist'] 
                dict_residtohept = dictionary_residtoheptad2(helices_info)
                a_dictionary, d_dictionary = get_residues2(knob_residues, dict_residtohept)
        all_a_dictionary[filename[0:4]]=a_dictionary
        all_d_dictionary[filename[0:4]]=d_dictionary
        socket_output.close
    else:
        pass

In [ ]:
#Run to quantify the number of pdb files used and the number of KIH interactions extracted
empty_a = []
for x in all_a_dictionary:
    if len(all_a_dictionary[x]) == 0:
        empty_a.append(x)

empty_d = []
for x in all_d_dictionary:
    if len(all_a_dictionary[x]) == 0:
        empty_d.append(x)

print('There are ' + str(len(empty_a)) + " pdfiles that don't contain KIH entries from the complete set of " + str(len(all_a_dictionary))
     + " (print empty_a or empty_d to get a list of the empty entries).")
counter_a = 0
counter_d = 0
for x in all_a_dictionary:
    for y in all_a_dictionary[x]:
        counter_a += 1
for x in all_d_dictionary:
    for y in all_d_dictionary[x]:
        counter_d += 1
print("From the combined set there are " + str(counter_a) + " KIH entries for a positions and " + str(counter_d) +
      " KIH entries for d postions.")

In [ ]:
#Use this code to quickly identify pdb files that have a specific hole residue motif
#  [0,1,2,3,4]  a =  aNterm, d, e, aCterm
#  [0,1,2,3,4]  d = dNterm, g, a, dCterm

all_pdbs_match = {}
for entry in all_a_dictionary:
    for hole in all_a_dictionary[entry]:
        if all_a_dictionary[entry][hole][0] == 'A' and all_a_dictionary[entry][hole][3] == 'E':
            all_pdbs_match[entry]='x'
        else:
            pass

__The following cells are used to produce frequencies from the data.__

In [ ]:
#Generate two dictionaries for a and d knob residues (a_positions and d_positions). 
#Keys represent the amino acid identity and the values is a nested dictionary where values represent 
#a running total of knob residues and values are lists of knob and hole single-letter-code amino acids

one_letter_code = {'A':'ALA','C':'CYS','D':'ASP','E':'GLU','F':'PHE','G':'GLY','H':'HIS','I':'ILE','K':'LYS',
                    'L':'LEU','M':'MET','N':'ASN','P':'PRO','Q':'GLN','R':'ARG','S':'SER','T':'THR','V':'VAL'               
                    ,'W':'TRP','Y':'TYR'}

three_letter_code = {'ALA':'A','CYS':'C','ASP':'D','GLU':'E','PHE':'F','GLY':'G','HIS':'H','ILE':'I','LYS':'K',
                    'LEU':'L','MET':'M','ASN':'N','PRO':'P','GLN':'Q','ARG':'R','SER':'S','THR':'T','VAL':'V'               
                    ,'TRP':'W','TYR':'Y'}

a_positions = {'ALA':{}, 'CYS':{}, 'ASP':{}, 'GLU':{}, 'PHE':{}, 'GLY':{}, 'HIS':{}, 'ILE':{}, 'LYS':{}, 
               'LEU':{}, 'MET':{}, 'ASN':{}, 'PRO':{}, 'GLN':{}, 'ARG':{}, 'SER':{}, 'THR':{}, 'VAL':{}, 
               'TRP':{}, 'TYR':{}}

d_positions = {'ALA':{}, 'CYS':{}, 'ASP':{}, 'GLU':{}, 'PHE':{}, 'GLY':{}, 'HIS':{}, 'ILE':{}, 'LYS':{}, 
               'LEU':{}, 'MET':{}, 'ASN':{}, 'PRO':{}, 'GLN':{}, 'ARG':{}, 'SER':{}, 'THR':{}, 'VAL':{}, 
               'TRP':{}, 'TYR':{}}


number_of_entries = 0
number_of_knobs = 0

for pdb_code, knoblist in all_a_dictionary.items():
    number_of_entries += 1
    for number , KIH in knoblist.items():
        a_positions[one_letter_code[KIH[0]]].update({'a'+str(number_of_knobs): KIH})
        number_of_knobs += 1

number_of_entries = 0
number_of_knobs = 0
        
for pdb_code, knoblist in all_d_dictionary.items():
    number_of_entries += 1
    for number , KIH in knoblist.items():
        d_positions[one_letter_code[KIH[0]]].update({'d'+str(number_of_knobs): KIH})
        number_of_knobs += 1

In [ ]:
#a =  aNterm, d, e, aCterm
#d = dNterm, g, a, dCterm
frequency_dictionary = {} 
for aa in a_positions:
    
    all_positions = {'aN':[], 'd':[], 'e':[], 'aC':[], 'dN':[], 'g':[], 'a':[], 'dC':[]} 
    aa_a_position = list(a_positions[aa].values())
    aa_d_position = list(d_positions[aa].values())
    
    for x in aa_a_position:
        all_positions['aN'].append(x[1])
        all_positions['d'].append(x[2])
        all_positions['e'].append(x[3])
        all_positions['aC'].append(x[4])

    for x in aa_d_position:
        all_positions['dN'].append(x[1])
        all_positions['g'].append(x[2])
        all_positions['a'].append(x[3])
        all_positions['dC'].append(x[4])
    
    for position in all_positions:
        frequency_dictionary[aa+'_'+position]=CountFrequency(all_positions[position])

In [ ]:
#Make arrays from dictionary of data. Such that each column represents the canonical amino acids organised alphabetically
#and the rows represent the hole positions.

array_all_a = {}
array_all_d = {}
a_holepositions = ['aN', 'd', 'e', 'aC']
d_holepositions = ['dN', 'g', 'a', 'dC']
one_letter_code_list = sorted(list(one_letter_code))
for x in list(three_letter_code.keys()):
    empty_array = np.zeros((4,20))
    position_counter = 0
    for pos in a_holepositions:
        if x+"_"+pos in frequency_dictionary:
            aminoacid_counter = 0
            for g in one_letter_code_list:
                if g in frequency_dictionary[x+"_"+pos]:
                    empty_array[position_counter,aminoacid_counter]=frequency_dictionary[x+"_"+pos][g]
                    aminoacid_counter +=1
                else:
                    aminoacid_counter +=1
        position_counter +=1
    array_all_a[x]=empty_array

for x in list(three_letter_code.keys()):
    ay = np.zeros((4,20))
    position_counter = 0
    for pos in d_holepositions:
        if x+"_"+pos in frequency_dictionary:
            aminoacid_counter = 0
            for g in one_letter_code_list:
                if g in frequency_dictionary[x+"_"+pos]:
                    ay[position_counter,aminoacid_counter]=frequency_dictionary[x+"_"+pos][g]
                    aminoacid_counter +=1
                else:
                    aminoacid_counter +=1
        position_counter +=1
    array_all_d[x]=ay     

array_all_a_outof1, array_all_a_outof1_sum = convert_to_probability(array_all_a)
array_all_d_outof1, array_all_d_outof1_sum = convert_to_probability(array_all_d)
a_total = sum([int(x) for x in array_all_a_outof1_sum.values()])
d_total = sum([int(x) for x in array_all_d_outof1_sum.values()])

In [ ]:
#Execute code to quickly check frequency of knob residues
for key, value in sorted(array_all_a_outof1_sum.items(), key=lambda item: item[1]):
    print("%s: %s" % (key, value))

In [ ]:
#Execute code to output all frequencies into csv files
csv_dir = "./frequency_csv/"
for residue in array_all_d_outof1.keys():
    DF = pd.DataFrame(array_all_d_outof1[residue], columns = list(array_all_d_outof1.keys()))
    DF = DF.transpose()
    DF.columns= d_holepositions
    DF.to_csv(csv_dir+"d_positions_"+residue+".csv")
for residue in array_all_a_outof1.keys():
    DF = pd.DataFrame(array_all_a_outof1[residue], columns = list(array_all_a_outof1.keys()))
    DF = DF.transpose()
    DF.columns= a_holepositions
    DF.to_csv(csv_dir+"a_positions_"+residue+".csv")

__The following cells allow the frequency data to be plotted as Sequence Logos.__

In [ ]:
cd ./all_ap_dimers_seqlog/

In [ ]:
#Generate Sequence logos from frequency data

for residue in array_all_a_outof1:
    #file = open(residue+"_aposition_"+str(array_all_a_outof1_sum[residue])+".png","wb") 
    if np.sum(array_all_a_outof1[residue][0]) == 0.0:
        pass
    else:
        aa = seqlogo.Ppm(array_all_a_outof1[residue], alphabet_type="AA")
        filename = str(residue)+"_aposition_"+str(array_all_a_outof1_sum[residue])+".png"
        seqlogo.seqlogo(aa, ic_scale = False, format = 'png', size = 'medium', color_scheme='chemistry', 
            filename=filename, xaxis=True)
        
for residue in array_all_d_outof1:
    #file = open(residue+"_aposition_"+str(array_all_a_outof1_sum[residue])+".png","wb") 
    if np.sum(array_all_d_outof1[residue][0]) == 0.0:
        pass
    else:
        aa = seqlogo.Ppm(array_all_d_outof1[residue], alphabet_type="AA")
        filename = str(residue)+"_dposition_"+str(array_all_d_outof1_sum[residue])+".png"
        seqlogo.seqlogo(aa, ic_scale = False, format = 'png', size = 'medium', color_scheme='chemistry', 
            filename=filename, xaxis=True)
    